## load the required libraries

In [39]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
sns.set_style('darkgrid')
warnings.filterwarnings('ignore')

In [122]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score,f1_score,mean_squared_error,mean_squared_log_error
import lightgbm as lgb

## read the files

In [40]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Data_Test.xlsx')

In [41]:
print(train.shape, test.shape)

(6019, 13) (1234, 12)


#### basic data analysis
##### null values treatement
##### duplicates
##### outlier treatment

In [5]:
train.head(2)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50


In [7]:
print(train.isna().sum())
print(test.isna().sum())

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 0
Engine                 10
Power                  10
Seats                  11
New_Price            1052
dtype: int64


In [8]:
train.describe(include=[np.object])

,Name,Location,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,New_Price
count,6019,6019,6019,6019,6019,6017,5983,5983,824
unique,1876,11,5,2,4,442,146,372,540
top,Mahindra XUV500 W8 2WD,Mumbai,Diesel,Manual,First,18.9 kmpl,1197 CC,74 bhp,95.13 Lakh
freq,49,790,3205,4299,4929,172,606,235,6


In [9]:
train.describe(exclude=[np.object])

,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,2013.358199,5.873838e+04,5.278735,9.479468
std,3.269742,9.126884e+04,0.808840,11.187917
min,1998.000000,1.710000e+02,0.000000,0.440000
25%,2011.000000,3.400000e+04,5.000000,3.500000
50%,2014.000000,5.300000e+04,5.000000,5.640000
75%,2016.000000,7.300000e+04,5.000000,9.950000
max,2019.000000,6.500000e+06,10.000000,160.000000


In [194]:
train.nunique()

Name                 1876
Location               11
Year                   22
Kilometers_Driven    3093
Fuel_Type               5
Transmission            2
Owner_Type              4
Mileage               442
Engine                146
Power                 372
Seats                   9
New_Price             540
Price                1373
dtype: int64

##### Combining the train test.. no need to perform same task again on test we will split it later.

In [77]:
df = train.append(test, ignore_index=True)

In [78]:
df.sample(2)

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,New_Price,Owner_Type,Power,Price,Seats,Transmission,Year
4362,814 CC,Petrol,27886,Jaipur,22.0 kmpl,Hyundai EON Magna,NaN,First,55 bhp,2.85,5.0,Manual,2016
519,1591 CC,Petrol,69434,Chennai,15.8 kmpl,Hyundai Creta 1.6 SX,14.84 Lakh,First,121.3 bhp,8.00,5.0,Manual,2016


In [79]:
df.dtypes

Engine                object
Fuel_Type             object
Kilometers_Driven      int64
Location              object
Mileage               object
Name                  object
New_Price             object
Owner_Type            object
Power                 object
Price                float64
Seats                float64
Transmission          object
Year                   int64
dtype: object

In [80]:
df['Mileage'].unique()

array(['26.6 km/kg', '19.67 kmpl', '18.2 kmpl', '20.77 kmpl', '15.2 kmpl',
       '21.1 km/kg', '23.08 kmpl', '11.36 kmpl', '20.54 kmpl',
       '22.3 kmpl', '21.56 kmpl', '16.8 kmpl', '25.2 kmpl', '12.7 kmpl',
       '0.0 kmpl', '13.5 kmpl', '25.8 kmpl', '28.4 kmpl', '20.45 kmpl',
       '14.84 kmpl', '22.69 kmpl', '23.65 kmpl', '13.53 kmpl',
       '18.5 kmpl', '14.4 kmpl', '20.92 kmpl', '17.5 kmpl', '12.8 kmpl',
       '19.01 kmpl', '14.53 kmpl', '11.18 kmpl', '12.4 kmpl',
       '16.09 kmpl', '14.0 kmpl', '24.3 kmpl', '18.15 kmpl', '11.74 kmpl',
       '22.07 kmpl', '19.7 kmpl', '25.4 kmpl', '25.32 kmpl', '14.62 kmpl',
       '14.28 kmpl', '14.9 kmpl', '11.25 kmpl', '24.4 kmpl', '16.55 kmpl',
       '17.11 kmpl', '22.9 kmpl', '17.8 kmpl', '18.9 kmpl', '15.04 kmpl',
       '25.17 kmpl', '20.36 kmpl', '13.29 kmpl', '13.68 kmpl',
       '20.0 kmpl', '15.8 kmpl', '25.0 kmpl', '16.4 kmpl', '24.52 kmpl',
       '22.1 kmpl', '8.5 kmpl', '15.1 kmpl', '16.95 kmpl', '19.64 kmpl',
       '16.

In [81]:
df['Mileage'] = df['Mileage'].apply(lambda x : str(x).split(' ')[0]).astype(float)

In [82]:
df['Engine'] = df['Engine'].apply(lambda x : str(x).split(" ")[0]).astype(float)

In [83]:
df['Power'].unique()

array(['58.16 bhp', '126.2 bhp', '88.7 bhp', '88.76 bhp', '140.8 bhp',
       '55.2 bhp', '63.1 bhp', '171.5 bhp', '103.6 bhp', '74 bhp',
       '103.25 bhp', '116.3 bhp', '187.7 bhp', '115 bhp', '175.56 bhp',
       '98.6 bhp', '83.8 bhp', '167.62 bhp', '190 bhp', '88.5 bhp',
       '177.01 bhp', '80 bhp', '67.1 bhp', '102 bhp', '108.45 bhp',
       '138.1 bhp', '184 bhp', '179.5 bhp', '103.5 bhp', '64 bhp',
       '82 bhp', '254.8 bhp', '73.9 bhp', '46.3 bhp', '37.5 bhp',
       '77 bhp', '82.9 bhp', '149.92 bhp', '138.03 bhp', '112.2 bhp',
       '163.7 bhp', '71 bhp', '105 bhp', '174.33 bhp', '75 bhp',
       '103.2 bhp', '53.3 bhp', '78.9 bhp', '147.6 bhp', '147.8 bhp',
       '68 bhp', '186 bhp', '170 bhp', '69 bhp', '140 bhp', '78 bhp',
       '194 bhp', '500 bhp', '108.5 bhp', '86.8 bhp', '187.74 bhp',
       'null bhp', '132 bhp', '86.7 bhp', '73.94 bhp', '117.3 bhp',
       '218 bhp', '168.5 bhp', '89.84 bhp', '110 bhp', '90 bhp',
       '82.85 bhp', '67 bhp', '241.4 bhp', '3

In [84]:
# found null bhpas value in the data
df['Power'] = df['Power'].replace('null bhp','0 bhp').apply(lambda x : str(x).split(' ')[0]).astype(float)

In [85]:
df['New_Price'] = df['New_Price'].fillna('-1 Lakhs').apply(lambda x: x.split(' ')[0]) 
df['New_Price'] = df['New_Price'].astype(float).replace(-1, np.nan)

In [86]:
df['Seats'] = df['Seats'].fillna(5)

In [87]:
df['Car_Age']=df['Year'].apply(lambda x: 2019-x)

In [88]:
df['Year'] = df['Year'].astype('category')
df['Location'] = df['Location'].astype('category')
df['Fuel_Type'] = df['Fuel_Type'].astype('category')
df['Transmission'] = df['Transmission'].astype('category')
df['Owner_Type'] = df['Owner_Type'].astype('category')

In [89]:
df.isnull().sum()

Engine                 46
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
New_Price            6247
Owner_Type              0
Power                  46
Price                1234
Seats                   0
Transmission            0
Year                    0
Car_Age                 0
dtype: int64

In [90]:
df['Name1'] = df['Name'].apply(lambda x: ' '.join(x.split(' ')[:2]))

# df['Engine'] = df.groupby(['Name1']).transform(lambda x: x.fillna(x.median()))['Engine']
# df['Power'] = df.groupby(['Name1']).transform(lambda x: x.fillna(x.median()))['Power']
# df['New_Price'] = df.groupby(['Name1']).transform(lambda x: x.fillna(x.mean()))['New_Price']

In [91]:
df['Engine']=df.groupby(['Name1']).transform(lambda x: x.fillna(x.mean(skipna=True)))['Engine']
# df['Engine'] = df.groupby(['Name1']).transform(lambda x: x.fillna(x.median()))['Engine']
df['Power']=df.groupby(['Name1']).transform(lambda x: x.fillna(x.mean(skipna=True)))['Power']
# df['Power'] = df.groupby(['Name1']).transform(lambda x: x.fillna(x.median()))['Power']
# df['Seats']=df.groupby(['Name1']).transform(lambda x: x.fillna(x.mean(skipna=True)))['Seats']
df['Seats'] = df.groupby(['Name1']).transform(lambda x: x.fillna(x.median()))['Seats']
df['Mileage']=df.groupby(['Name1']).transform(lambda x: x.fillna(x.mean(skipna=True)))['Mileage']
# df['Mileage'] = df.groupby(['Name1']).transform(lambda x: x.fillna(x.median()))['Mileage']
df['New_Price']=df.groupby(['Name1']).transform(lambda x: x.fillna(x.mean(skipna=True)))['New_Price']

In [92]:
df.isnull().sum()

Engine                  0
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
New_Price            1512
Owner_Type              0
Power                   0
Price                1234
Seats                   0
Transmission            0
Year                    0
Car_Age                 0
Name1                   0
dtype: int64

In [93]:
df['Name2'] = df['Name'].apply(lambda x: x.split(' ')[0])

df['New_Price'] = df.groupby(['Name2']).transform(lambda x: x.fillna(x.mean()))['New_Price']

In [94]:
df.isnull().sum()

Engine                  0
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
New_Price             162
Owner_Type              0
Power                   0
Price                1234
Seats                   0
Transmission            0
Year                    0
Car_Age                 0
Name1                   0
Name2                   0
dtype: int64

In [95]:
df.dtypes

Engine                float64
Fuel_Type            category
Kilometers_Driven       int64
Location             category
Mileage               float64
Name                   object
New_Price             float64
Owner_Type           category
Power                 float64
Price                 float64
Seats                 float64
Transmission         category
Year                 category
Car_Age                 int64
Name1                  object
Name2                  object
dtype: object

In [102]:
#### Apply genral statistics numeric features 

In [96]:
agg_col={'Seats':['mean','sum','min','max'],'Car_Age':['mean','sum','min','max'],'Kilometers_Driven':['mean','sum','min','max'],'Mileage':['mean','sum','min','max'],'New_Price':['mean','sum','min','max']
          ,'Power':['mean','sum','min','max'],'Engine':['mean','sum','min','max'],'Location':['count']}
df1=df.copy()

df2=df.groupby(['Name1']).agg(agg_col).reset_index()
df2.columns=['_'.join(col).rstrip('_') for col in [c[::-1] for c in df2.columns.values]]
df2.rename(columns={"_Name1": "Name1"},inplace =True)

df1=df.merge(df2,on=['Name1'],how='left')


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [97]:
df1.head(2)

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,New_Price,Owner_Type,Power,Price,...,min_New_Price,max_New_Price,mean_Power,sum_Power,min_Power,max_Power,mean_Kilometers_Driven,sum_Kilometers_Driven,min_Kilometers_Driven,max_Kilometers_Driven
0,998.0,CNG,72000,Mumbai,26.60,Maruti Wagon R LXI CNG,5.469556,First,58.16,1.75,...,4.59,6.80,66.285198,11798.765198,57.5,81.8,53742.988764,9566252,600,200000
1,1582.0,Diesel,41000,Pune,19.67,Hyundai Creta 1.6 CRDi SX Option,16.415000,First,126.20,12.50,...,11.81,19.18,121.830508,14376.000000,88.7,126.2,40361.211864,4762623,2500,101065


#### Manually filled these data from business rules

In [98]:
df[(df['Seats'].isna())]
df.loc[(df['Seats'].isna() ),'Seats']=5
# df.loc[((df['Mileage'].isna() ) & (df['Name1']=="Toyota Prius")),'Mileage']=24

In [99]:
df[(df['Mileage'].isna())]
df.loc[((df['Mileage'].isna() ) & (df['Name1']=="Mahindra E")),'Mileage']=140
df.loc[((df['Mileage'].isna() ) & (df['Name1']=="Toyota Prius")),'Mileage']=24

In [100]:
df.isnull().sum()

Engine                  0
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 0
Name                    0
New_Price             162
Owner_Type              0
Power                   0
Price                1234
Seats                   0
Transmission            0
Year                    0
Car_Age                 0
Name1                   0
Name2                   0
dtype: int64

In [101]:
# df.loc[((df['Mileage'].isna() ) & (df['Name1']=="Mahindra E")),'Mileage']
# New_Price
# df[(df['New_Price'].isna())]

In [103]:
print (df.shape,df1.shape)

(7253, 16) (7253, 45)


In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf1 = TfidfVectorizer(lowercase=True, stop_words='english', ngram_range=(1, 1), max_df=1.0, min_df=2, binary=True)
df_name = tf1.fit_transform(df1['Name'])
df_name = pd.DataFrame(data=df_name.toarray(), columns=tf1.get_feature_names())

df = pd.concat([df1, df_name], axis=1) 

In [108]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.head(2)

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,New_Price,Owner_Type,Power,Price,Seats,Transmission,Year,Car_Age,mean_Engine,sum_Engine,min_Engine,max_Engine,count_Location,mean_Mileage,sum_Mileage,min_Mileage,max_Mileage,mean_Seats,sum_Seats,min_Seats,max_Seats,mean_Car_Age,sum_Car_Age,min_Car_Age,max_Car_Age,mean_New_Price,sum_New_Price,min_New_Price,max_New_Price,mean_Power,sum_Power,min_Power,max_Power,mean_Kilometers_Driven,sum_Kilometers_Driven,min_Kilometers_Driven,max_Kilometers_Driven,05,0d,0l,10,100,110,110ps,118d,16,177,1l,200,2001,2003,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2019,20d,220,220d,230,230jm,250,250d,280,2l,2p,2t,2wd,30,300,30d,320,320d,320i,325i,330d,35,350,350d,40d,43,45,4d,4l,4matic,4wd,4x2,4x4,500,520d,523i,525d,525i,530d,530i,5d,5l,5str,640d,650i,6l,730ld,740li,75ps,7s,800,85,85ps,8s,99,a180,a200,a3,a4,a6,a7,a8,abs,ac,accent,accord,active,advantage,adventure,aero,ags,airbag,aktiv,alloy,alpha,altis,alto,amaze,ambiente,ambition,ameo,amg,amt,anniversary,aqua,aspire,asta,attraction,audi,aura,automatic,avantgarde,avantgare,avantgrade,aveo,avn,avventura,aw,awd,b180,b200,b3,baleno,base,beat,bentley,benz,bhp,black,blue,bmw,bolero,bolt,br,brezza,brio,brv,bs,bs3,bsii,bsiii,bsiv,c2,c250,c8,camry,captiva,captur,carbon,cargo,cayenne,cayman,cdi,cedia,celebration,celerio,cgi,chevrolet,ciaz,city,civic,cla,class,classic,climber,club,clubman,clxi,cng,comfortline,compass,convertible,cooper,corolla,corporate,country,countryman,coupe,cr,crde,crdi,creta,cross,crosspolo,cruze,crysta,crz,cs,cvt,cx,d2,d3,d4,d4dg,d5,d6,d75,datsun,ddis,delta,design,...,multijet,multitronic,n6,nano,nav,new,nexon,nissan,nuvosport,nxt,octavia,omni,opt,option,optional,optra,outlander,pack,package,pajero,panamera,paseo,passat,pd,petrol,platine,platinum,plus,polo,porsche,portfolio,pre,premium,prestige,primo,progressive,ps,pulse,punto,pure,q3,q5,q7,quadrajet,quadrajet90,qualis,quanto,quattro,r350,range,rapid,redi,renault,revotorq,revotron,rexton,rider,ritz,roof,rover,rs,rs5,rvi,rx5,rx7,rxe,rxl,rxt,rxz,s10,s4,s6,s60,s8,s80,safari,safety,safire,sail,santa,santro,scala,scorpio,sd4,sdrive,sdrive20d,sdv8,se,seater,seats,sedan,select,series,sfx,sh,shvs,sigma,signature,skoda,slc,sle,slk,slx,sonata,sp,spark,special,speed,sport,sportivo,sportline,sports,sportz,spotz,spur,ssangyong,standard,star,std,stingray,storme,str,style,summum,sumo,sun,sunny,sunroof,superb,sv,swift,sx,sx4,sxi,t8,tata,tavera,tc,tcdi,td4,td6,tdci,tdi,tdv6,tdv8,teana,technology,terra,terrano,tfsi,thar,ti,tiago,tigor,tiptronic,titanium,tone,tour,toyota,trail,transform,trd,trend,trendline,tsi,tt,tucson,turbo,tuv,twist,ultima,v2,v40,v6,v8,va,vct,vd,vdi,vento,verito,verna,versa,vgt,vista,vitara,vl,vlx,vogue,volkswagen,volvo,vtec,vtech,vti,vtvt,vvt,vx,vxd,vxi,w10,w4,w6,w8,wagon,wheel,wo,wrv,x1,x3,x5,x6,xc60,xc90,xcent,xdrive,xdrive20d,xdrive30d,xe,xenon,xf,xg,xi,xing,xj,xl,xline,xlt,xm,xms,xo,xp,xt,xta,xuv500,xv,xxi,xylo,xz,xza,yeti,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,998.0,CNG,72000,Mumbai,26.60,5.469556,First,58.16,1.75,5.0,Manual,2010,9,1015.180791,180702.180791,998.0,1197.0,178,20.915674,3722.99,12.0,33.54,5.0,890.0,5.0,5.0,6.966292,1240,0,18,5.469556,973.580889,4.59,6.80,66.285198,11798.765198,57.5,81.8,53742.988764,9566252,600,200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.641471,0.0,0.0,0.0,0.0,

In [105]:
df.drop(['Name','Name1','Name2'], axis=1, inplace=True)

In [106]:
train_df = df[df['Price'].isnull()!=True]
test_df = df[df['Price'].isnull()==True]
test_df.drop('Price', axis=1, inplace=True)

In [107]:
train_df.shape, test_df.shape

((6019, 696), (1234, 695))

In [113]:
train_df['Price'] = np.log1p(train_df['Price'])


### split ji Train test

In [114]:
X = train_df.drop(labels=['Price'], axis=1)
y = train_df['Price'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)

In [115]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((4514, 695), (4514,), (1505, 695), (1505,))

## Model Trainign

In [116]:
from math import sqrt 
from sklearn.metrics import mean_squared_log_error

In [131]:

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)


lgbm=LGBMRegressor(boosting_type='gbdt',
#     num_leaves=255,
#     max_depth=-1,
    learning_rate=0.001,
    n_estimators=6000,
#     subsample_for_bin=200000,
#     objective=None,
#     class_weight=None,
#     min_split_gain=0.0,
#     min_child_weight=0.001,
    min_child_samples=100
                ,
#     subsample=1.0,
#     subsample_freq=1,
    colsample_bytree=0.4,
#     reg_alpha=0.5,
#     reg_lambda=1.0,
    random_state=1994,
    n_jobs=10,
    silent=True)
# X_train,X_val,y_train,y_val
# train_X, val_X, train_y, val_y

lgbm.fit(train_X,train_y,eval_set=[(val_X, val_y)],eval_metric='rmse', early_stopping_rounds=100,verbose=100)
out=lgbm.predict(val_X)
print(rmsle_lgb(val_y,out))
print('RMSLE:', sqrt(mean_squared_log_error(np.expm1(val_y), np.expm1(out))))




Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.46281	valid_0's rmse: 0.680302
[200]	valid_0's l2: 0.394297	valid_0's rmse: 0.627931
[300]	valid_0's l2: 0.336472	valid_0's rmse: 0.580062
[400]	valid_0's l2: 0.288101	valid_0's rmse: 0.53675
[500]	valid_0's l2: 0.247847	valid_0's rmse: 0.497842
[600]	valid_0's l2: 0.214104	valid_0's rmse: 0.462714
[700]	valid_0's l2: 0.186223	valid_0's rmse: 0.431536
[800]	valid_0's l2: 0.162582	valid_0's rmse: 0.403214
[900]	valid_0's l2: 0.14273	valid_0's rmse: 0.377796
[1000]	valid_0's l2: 0.12624	valid_0's rmse: 0.355302
[1100]	valid_0's l2: 0.112237	valid_0's rmse: 0.335018
[1200]	valid_0's l2: 0.100493	valid_0's rmse: 0.317006
[1300]	valid_0's l2: 0.0904554	valid_0's rmse: 0.300758
[1400]	valid_0's l2: 0.0821661	valid_0's rmse: 0.286646
[1500]	valid_0's l2: 0.0751177	valid_0's rmse: 0.274076
[1600]	valid_0's l2: 0.0690062	valid_0's rmse: 0.26269
[1700]	valid_0's l2: 0.0638086	valid_0's rmse: 0.252604
[1800]	vali

### Predict on test set

In [133]:
Xtest = test_df

In [134]:

testO=lgbm.predict(Xtest)

In [137]:
finalDf=pd.DataFrame({'Price':np.expm1(testO)})
finalDf.to_csv('output.csv',index=False)
finalDf.head()

,Price
0,2.646954
1,2.716291
2,16.580175
3,4.022373
4,4.759327
